In [1]:
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB

import datetime as dt

In [2]:
data = pd.read_parquet("stock_data.parquet")
stock_names = data['comnam'].unique()

# repeat = True
# while repeat:
#     count = 0
#     dfs = []
#     sizes = []
#     relevant_stocks = np.random.choice(stock_names,size=10, replace=False)
#     for stock in relevant_stocks:
#         subset = data[data['comnam'] == stock]
#         df = subset[['prc']]
#         df.columns = [stock]
#         df.index = pd.to_datetime(subset['date'])
#         dfs.append(df)
#         sizes.append(df.shape[0])
#     dfs = [x for _,x in reversed(sorted(zip(sizes,dfs), key = lambda x:x[0]))]
#     final = dfs[0]
#     for i in range(1, 10):
#         final = final.join(dfs[i])
#     final.index.name = 'date'
#     final = final.groupby(by='date').mean()
#     final = final.interpolate()
#     period = np.random.randint(30, final.shape[0])
#     final = final.iloc[period-30:period]
#     if final.iloc[0].isna().any() or final.iloc[29].isna().any():
#         repeat = True
#     else:
#         repeat = False

In [3]:
# Remove stocks with na in price column random stocks
# TODO: Remove the complete stock if any na
data_clean = data.dropna(axis=0, subset=['prc'])

# Pick 10 stocks at random
relevant_stocks = np.random.choice(data_clean.comnam.unique(),size=10, replace=False)
data_filt = data_clean.query("comnam in @relevant_stocks")

# Pivot data (columns: stock name, index: date)
data_pivot = data_filt.pivot_table(index="date", values="prc", columns="comnam")

# Focus on window of 30 trading days with random start dt
start_dt = np.random.choice(data_pivot.index[:-30],size=1, replace=False)[0]
final = data_pivot.query("date >= @start_dt")
final = final.iloc[:30]

final

comnam,AUTODESK INC,CENTERSPACE,DIGITAL REALTY TRUST INC,ENERSYS,PIMCO DYNAMIC CR & MORT INC FD,PROSPERITY BANCSHARES INC,PRUDENTIAL REINSURANCE HOLD INC,SANMINA SCI CORP,SELIGMAN PREMIUM TECH GROWTH FD,U S X US STEEL GROUP INC
date,,,,,,,,,,
2021-03-15,273.019989,73.389999,135.070007,101.879997,22.570000,80.370003,255.970001,40.730000,30.400000,23.139999
2021-03-16,275.010010,72.599998,135.679993,101.250000,22.540001,78.769997,253.369995,41.369999,30.610001,21.309999
2021-03-17,272.839996,71.150002,134.679993,102.550003,22.490000,79.010002,251.279999,42.639999,30.900000,22.040001
2021-03-18,263.440002,72.010002,134.160004,101.250000,22.350000,79.959999,251.949997,41.669998,30.240000,22.650000
2021-03-19,261.500000,70.870003,133.509995,99.389999,22.440001,79.230003,245.830002,41.049999,30.260000,22.410000
2021-03-22,265.959991,69.360001,136.550003,97.860001,22.520000,76.540001,243.880005,41.009998,30.719999,21.790001
2021-03-23,269.000000,69.769997,140.000000,93.290001,22.290001,74.150002,241.360001,39.490002,30.370001,19.860001
2021-03-24,263.179993,68.730003,140.089996,90.790001,22.350000,73.790001,242.949997,40.000000,30.000000,19.570000
2021-03-25,262.190002,68.400002,138.360001,91.599998,22.350000,75.800003,247.860001,40.529999,29.910000,20.370001


Create an initial portfolio

In [21]:
def get_prices_window(start_dt):
    final = data_pivot.query("date >= @start_dt")
    final = final.iloc[:30]
    return final

def generate_portfolios(stock_prices, init_budget=100000):
    initial_budget = init_budget
    initial_prices = stock_prices.iloc[0].to_numpy()
    available_money = initial_budget * .95
    P_0 = np.zeros(10) #initial portfolio
    break_bool = True
    ignore =  np.random.randint(0,10)
    while break_bool:
        stock_buy = np.random.randint(0,10)
        price = initial_prices[stock_buy]
        if stock_buy != ignore:
            if (available_money * .75) - price >= 0:
                available_money = available_money - price
                P_0[stock_buy] = P_0[stock_buy] + 1
            else:
                break_bool = False

    w_0 = initial_budget * .05 + available_money
    print(w_0)
    print(P_0)
    final_prices = stock_prices.iloc[29].to_numpy()
    available_money = initial_budget * .9
    break_bool = True
    P_t = np.zeros(10) #target portfolio

    while break_bool:
        stock_buy = np.random.randint(0,10)
        price = final_prices[stock_buy]
        if available_money - price >= 0:
            available_money = available_money - price
            P_t[stock_buy] = P_t[stock_buy] + 1
        else:
            break_bool = False
    print(P_t)
    return P_0, P_t, w_0

# Formulations

## With Perfect Information

In [16]:
def get_naive_opt(P_0, P_t, w_0, initial_budget, initial_prices, final_prices):
    F = 9.95 * np.ones(10)# trading fees
    # P_0 = starting portfolio
    # P_t = target portfolio
    # w_0 = starting cash
    # w_f = final free cash
    budget = 20000
    M = initial_budget



    m = gp.Model()
    # Define Variables
    P_buy = m.addMVar((10,), vtype=GRB.INTEGER)
    P_sell = m.addMVar((10,), vtype=GRB.INTEGER)
    z_sell = m.addMVar((10,), vtype=GRB.BINARY)
    z_buy = m.addMVar((10,), vtype=GRB.BINARY)
    # Constraints

    ## Meet Target
    m.addConstr(
        P_0 + P_buy - P_sell >= P_t
    )

    ## Have Money
    m.addConstr(
        w_0 + initial_prices @ P_sell - initial_prices @ P_buy - F @ z_sell - F @ z_buy >=0
    )

    ## if we sell a stock, we pay a trading price
    m.addConstr(
        M*(1-z_sell) >= P_sell
    )

    ## If we buy a stock, we pay a trading fee

    m.addConstr(
        M*(1-z_buy) >= P_buy
    )

    free_cash = w_0 + initial_prices @ P_sell - initial_prices @ P_buy
    # Set objective: Maximize end portfolio value
    m.ModelSense = GRB.MAXIMIZE
    m.setObjective(
        free_cash - F @ z_sell - F @ z_buy + final_prices @ (P_0 + P_buy - P_sell)
    )

    m.optimize()
    final_portfolio = P_0 + P_buy.x - P_sell.x
    free_cash = w_0 + initial_prices @ P_sell.x - initial_prices @ P_buy.x
    final_portfolio_value = free_cash + final_prices @ final_portfolio
    print(f"Final Portfolio Value: {final_portfolio_value}")
    print(f"Free Cash: {free_cash}")
    print(f"Final Portfolio: {final_portfolio}")
    return final_portfolio_value, final_portfolio, free_cash

## Multi-stage

##

In [8]:
def get_multi_stage_opt(P_0, P_t, w_0, initial_budget, final):    
    F = 9.95  * np.ones(10)# trading fees
    prices = final.interpolate().to_numpy()
    M = initial_budget
    # P_0 = starting portfolio
    # P_t = target portfolio
    # w_0 = starting cash

    m = gp.Model()

    # Define Variables
    P_buy = m.addMVar((30,10), vtype=GRB.INTEGER)
    P_sell = m.addMVar((30,10), vtype=GRB.INTEGER)
    z_sell = m.addMVar((30,10), vtype=GRB.BINARY)
    z_buy = m.addMVar((30,10), vtype=GRB.BINARY)
    P_time = m.addMVar((30,10), vtype=GRB.INTEGER)
    w = m.addMVar((30,), vtype=GRB.CONTINUOUS, lb = 0)
    # Constraints

    ## Represent Portfolios across time

    ### Time step 0
    m.addConstr(
        P_time[0,:] == P_0 + P_buy[0,:] - P_sell[0,:]
    )
    ### Rest of the time
    m.addConstrs((
        P_time[i,:] == P_time[i-1,:] - P_buy[i,:] - P_sell[i,:] for i in range(1,30)
    ))

    ### Final_target
    m.addConstr(
        P_time[29,:] >= P_t
    )

    ## Represent money
    ### First time step
    m.addConstr(
        w[0] == w_0 - prices[0,:] @ P_buy[0,:] + prices[0,:] @ P_sell[0,:] - F @ z_buy[0,:] - F @ z_sell[0,:]
    )
    ## Rest of the time
    m.addConstrs((
        w[i] == w[i-1] - prices[i,:] @ P_buy[i,:] + prices[i,:] @ P_sell[i,:] - F @ z_buy[i,:] - F @ z_sell[i,:] for i in range(1,30)
    ))

    ## Trading fees
    ### if we sell a stock, we pay a trading price
    m.addConstr(
        M*(1-z_sell) >= P_sell
    )

    ### If we buy a stock, we pay a trading fee
    m.addConstr(
        M*(1-z_buy) >= P_buy
    )

    # Maximize end Portfolio value
    m.ModelSense = GRB.MAXIMIZE
    m.setObjective(
        w[29] + prices[29,:] @ P_time[29,:]
    )
    m.optimize()
    final_portfolio = P_time[29,:].x
    free_cash = w[29].x 
    final_portfolio_value = free_cash + prices[29,:] @ final_portfolio
    print(f"Final Portfolio Value: {final_portfolio_value}")
    print(f"Free Cash: {free_cash}")
    print(f"Final Portfolio: {final_portfolio}")
    return final_portfolio_value, final_portfolio, free_cash

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[x86])

CPU model: Intel(R) Core(TM) i9-9880H CPU @ 2.30GHz
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 940 rows, 1530 columns and 3659 nonzeros
Model fingerprint: 0xb4be9273
Variable types: 30 continuous, 1500 integer (600 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+05]
  Objective range  [1e+00, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [6e+01, 1e+05]
Presolve removed 305 rows and 307 columns
Presolve time: 0.01s
Presolved: 635 rows, 1223 columns, 3039 nonzeros
Variable types: 30 continuous, 1193 integer (595 binary)

Root relaxation: objective 1.054780e+05, 16 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 105477.952    0    2          - 105477.952      -     -    0s
H    0     0

In [ ]:
simulations = 1000
rng = np.random.default_rng()
budget_vec = rng.integers(low=1000, high=10000, size=simulations)
start_dt_vec = np.random.choice(data_pivot.index[:-30], size=simulations, replace=True)

# Could parallelize this but fuck it
for i, budget in enumerate(budget_vec):
    stock_price_window = get_prices_window(start_dt_vec[i])
    P_0, P_t, w_0 = generate_portfolios(stock_prices = stock_price_window, init_budget=budget)
